# Logistic Regression Consulting Project

## Solutions

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [0]:
data = spark.read.csv('dbfs:/FileStore/shared_uploads/arunyuvi0206@gmail.com/customer_churn-1.csv',inferSchema=True,
                     header=True)

In [0]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



### Check out the data

In [0]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [0]:
data.columns

Out[7]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Format for MLlib

We'll ues the numerical columns. We'll include Account Manager because its easy enough, but keep in mind it probably won't be any sort of a signal because the agency mentioned its randomly assigned!

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [0]:
output = assembler.transform(data)

In [0]:
final_data = output.select('features','churn')

### Test Train Split

In [0]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Fit the model

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
lr_churn = LogisticRegression(labelCol='churn')

In [0]:
fitted_churn_model = lr_churn.fit(train_churn)

In [0]:
training_sum = fitted_churn_model.summary

In [0]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                634|                634|
|   mean|  0.167192429022082|0.12302839116719243|
| stddev|0.37344221469338706|0.32872914728273933|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [0]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.79944549677498...|[0.68985585501782...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.33168027946692...|[0.99518733628544...|       0.0|
|[28.0,11204.23,0....|    0|[1.78799829768773...|[0.85668168659020...|       0.0|
|[28.0,11245.38,0....|    0|[3.34537911214332...|[0.96595319260181...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.06355855941046...|[0.98310268030637...|       0.0|
|[29.0,9378.24,0.0...|    0|[4.44635361543177...|[0.98841456627363...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.10036640306740...|[0.98370337549358...|       0.0|
|[29.0,11274.46,1....|    0|[4.48618116829476...|[0.98886187957793...|       0.0|
|[29.0,13240.01,1....|    0|[6.55897328665032...|[0.99858466584431...|       0.0|
|[30.0,7960.64,1

### Using AUC

In [0]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [0]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [0]:
auc

Out[23]: 0.7705773955773957

[Common question - what is a good AUC value?](https://stats.stackexchange.com/questions/113326/what-is-a-good-auc-for-a-precision-recall-curve)

### Predict on brand new unlabeled data

We still need to evaluate the new_customers.csv file!

In [0]:
final_lr_model = lr_churn.fit(final_data)

In [0]:
new_customers = spark.read.csv('dbfs:/FileStore/shared_uploads/arunyuvi0206@gmail.com/new_customers.csv',inferSchema=True,
                              header=True)

In [0]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [0]:
test_new_customers = assembler.transform(new_customers)

In [0]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
final_results = final_lr_model.transform(test_new_customers)

In [0]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Ok! That is it! Now we know that we should assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!

## Great Job!